# GNN и GCN 

Существует несколько библиотек для работы с GNN, я же остановился на ```torch_geometric```, потому что я хорошо знаком с обычным ```torch```. Здесь будет представлена вводная информация по работе с этой библиотекой и по работе с GNN в целом. Я пытался описать несколько уровней асбстракции и шел от высокой к низкой, по заветам [@jeremyphoward](https://twitter.com/jeremyphoward). Оказалось, что это сложнее, чем я думал -- вот и повод больше ценить его курс.

## Особенности работы с графом в ```torch_geometric```

### ```torch_geometric.data.Data```

Cразу же начнем с кода — в библиотеке ```torch_geometric``` для выражения графа выделен отдельный класс ```torch_geometric.data.Data```, имеющий следующие параметры:
* ```data.x``` -- матрица признаков каждой вершины
* ```data.edge_index``` -- ребра в графе
* ```data.eage_attr``` -- атрибуты ребер
* ```data.y``` -- таргет
* ```data.pos``` -- позиции вершин

Для инициализации ```Data``` не нужно указывать всю эту информацию, как будет показано ниже.

<p>Попробуем создать простой граф:</p>
<img src="./media/graph.svg" width=500px>

Начнем с импорта:

In [6]:
import torch
from torch_geometric.data import Data

Нам понадобится способ описать в виде тензора ребра, самый экономный вариант -- записать их в виде пар:

In [7]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)

Однако такой формат не пойдет. Тензор должен иметь размер ```[2, num_edges]```, то есть или ```edge_index``` придется транспонировать отдельно (а затем ещё и, по хорошему, вызывать ```contiguous()```), или приучить себя сразу писать в такой форме:

In [8]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

Далее нам потребуется тензор для описания признаков каждой вершины. Важно: ```x.dim() == 2```, так как ```x``` должен иметь размер ```[num_nodes, num_dimensions]```

In [9]:
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

Собрав все вместе получаем:

In [10]:
data = Data(x=x, edge_index=edge_index)
data

Data(edge_index=[2, 4], x=[3, 1])

### Датасеты: Cora 

От единичных графов к датасетам, возьмем для примера Cora. Заодно попробуем построить первую модель с использованием ```GCN```, пока не углубляясь в принципы её работы. Но сначала добавим немного контекста данным. 

Cora — это датасет, в котором вершины представляют научные статьи, а ребра — ссылки на эти статьи. Каждая вершина выражается 1433 призаками и имеет один из 7 классов. Фактически эти 1433 признака ничто иное как bag-of-words репрезентация названия. Важно понять, что создатели пошли на условность — ведь в конкретно этом случае $a_{ij}=a_{ji}=1$. 

In [11]:
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

Загрузка датасета. Заметьте, что мы также можем применять дополнительные трансформации к данным:

In [13]:
dataset = Planetoid(root='./data', name='Cora', transform=T.NormalizeFeatures())

В данном случае мы нормализовали ```dataset.data.x``` по строкам. Взглянем на объект класса ```Data``` -- атрибут датасета:

In [14]:
data = dataset.data
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

Здесь дела обстоят интереснее, чем в нашем игрушечном графе выше. На весь датасет -- один граф с 2708 вершинами (это видно по ```x```), предсказания нужно делать на уровне вершин, ведь таргетов у нас тоже 2708 (это видно по ```y```). Тренировачные, валидационные и тестовые данные находятся в этом же графе -- для них существуют свои маски со значениями ```True/False```. Ребер же в датасете -- 5278 (10556 / 2).

## Модель с GCN слоями  

In [16]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True)
        self.conv2 = GCNConv(16, dataset.num_classes, cached=True)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

Все как всегда -- ```nn.Module```, ```forward```, разница лишь в наличии новых слоев ```GCNConv``` -- первый с 16 выходными каналами, второй -- с 7. Похоже на обычный линейный слой. ```forward``` на вход будет принимать весь граф, внутри себя вычленять матрицу признаков и матрицу ребер, пропускать через конволюционные слои, функции активации и Dropout -- очень похоже на обычную модель.

Как обычно будет протекать и обучение: 

In [17]:
device='cuda'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)

best_val_acc = 0
for epoch in range(0, 150):
    
    model.train()
    optimizer.zero_grad()
    t = model(data)
    train_loss = F.nll_loss(t[data.train_mask], data.y[data.train_mask])
    train_loss.backward()
    optimizer.step()
    
    
    model.eval()
    accs = []
    for mask in [data.train_mask, data.val_mask]:
        pred = t[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    
    val_loss = F.nll_loss(t[data.val_mask], data.y[data.val_mask])
    train_acc, val_acc = accs
        
    if val_acc > best_val_acc:
        best_val_acc = val_acc
    
    if (epoch) % 10 == 0:
        log = 'Epoch: {:03d}, Train: {:.4f}, Train_loss: {:.4f} Val: {:.4f} Val_loss: {:.4f}'
        print(log.format(epoch, train_acc, train_loss, val_acc, val_loss))

Epoch: 000, Train: 0.2214, Train_loss: 1.9504 Val: 0.1820 Val_loss: 1.9398
Epoch: 010, Train: 0.9286, Train_loss: 0.7636 Val: 0.6840 Val_loss: 1.2537
Epoch: 020, Train: 0.9714, Train_loss: 0.2328 Val: 0.7280 Val_loss: 0.8889
Epoch: 030, Train: 0.9857, Train_loss: 0.1080 Val: 0.6980 Val_loss: 0.9714
Epoch: 040, Train: 1.0000, Train_loss: 0.0571 Val: 0.7080 Val_loss: 0.9899
Epoch: 050, Train: 1.0000, Train_loss: 0.0418 Val: 0.7320 Val_loss: 0.8939
Epoch: 060, Train: 1.0000, Train_loss: 0.0407 Val: 0.7480 Val_loss: 0.9238
Epoch: 070, Train: 1.0000, Train_loss: 0.0435 Val: 0.7280 Val_loss: 0.8863
Epoch: 080, Train: 1.0000, Train_loss: 0.0310 Val: 0.7280 Val_loss: 0.9531
Epoch: 090, Train: 1.0000, Train_loss: 0.0474 Val: 0.7040 Val_loss: 0.9445
Epoch: 100, Train: 0.9929, Train_loss: 0.0359 Val: 0.7420 Val_loss: 0.9032
Epoch: 110, Train: 1.0000, Train_loss: 0.0239 Val: 0.7100 Val_loss: 0.9895
Epoch: 120, Train: 0.9857, Train_loss: 0.0512 Val: 0.7220 Val_loss: 0.8958
Epoch: 130, Train: 1.0000

Результат на тестовых данных:

In [18]:
t = model(data)[data.test_mask]
F.nll_loss(t, data.y[data.test_mask]).item(), t.max(1)[1].eq(data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()

(0.6377240419387817, 0.804)

Могло бы быть и лучше, на [Papers with code](https://paperswithcode.com/sota/node-classification-on-cora) ```GCN``` дает результат в 81.5%. Однако пример выше служит другим целям -- я хотел показать как просто можно написать графовую сеть. Правда, это просто ровно до тех пор пока вы не решите углубиться.

## Немного математики и ```GCNConv``` 

```GCNConv``` впервые появился в статье [Semi-supervised Classification with Graph Convolutional Networks](https://arxiv.org/pdf/1609.02907.pdf). Взглянем на довольно пугающую формулу для одной вершины:
$$x'_i = \Theta \sum_{\mathcal{N(i) \cup \{i\}}} \frac{e_{i,j}}{\sqrt{\hat d_i \hat d_j}}x_j$$
...что легко можно интерпретировать как ужас во плоти. Но если отбросить самое страшное то получится что-то вроде:
$$x'_i = \Theta \sum_{j} c(i, j)x_j$$
То есть взвешенная сумма признаков перемноженная с параметрами $\Theta$. Ещё чуть-чуть и будет похоже на обычное произведение матрицы признаков на линейный слой. Теперь постепенно будем возвращать страшные штуки и начнем с критериев суммы.

$\mathcal{N(i)}$ есть ничто иное как соседи вершины $i$ (то есть вершины имеющие общее ребро с $i$). Приписка $\cup \{i\}$ значит "включая вершину $i$". Иными словами, мы взвешенно суммируем признаки соседей и самой вершины.

Теперь $c(i, j)$. Числитель $e_{i,j}$ есть вес ребра (по умолчанию = 1.0). $\hat d_j = 1 + \sum_{j\in\mathcal{N(i)}} e_{i,j}$, то есть в случае по умолчанию (c единичным значением веса ребра) $\hat d_j$ значит 1 + количество соседей вершины $j$.

Ну а теперь в матричном виде:  $$X' = \hat D ^ {-0.5} \hat A \hat D ^ {-0.5} X \Theta$$ где $\hat A = A + I$. $A$ - матрица смежности графа, $I$ - единичная матрица, а значит $\hat A$ - матрица смежности с петлями (петля - ребро, начало и конец которого находятся в одной и той же вершине). Матрица $\hat D$ инициализируется как $D_{ii} = \sum_{j=0}\hat A_{ij}$, то есть диагональная матрица.

Возьмем первую формулу и успростим её (основываясь на предположении что вес ребер = 1.0). Получится:
$$x_i'= \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\hat d_i \hat d_j}}  \left( {\Theta} x_j \right)$$

### ```MessagePassing```

Перед имплементацией поговорим о базовом классе ```MessagePassing``` (то же что и ```nn.Module```, но для графов). В его основе лежит форммула для обобщения GNN слоев: 

$$x_i^{(k)} = \gamma^{(k)} \left( x_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(x_i^{(k-1)}, x_j^{(k-1)},e_{j,i}\right) \right)
$$

Она тоже страшная, и для нее выделен отдельный раздел ниже. Пока что небольшие пояснения:
* $\square$ -- дифференцируемая, независимая от порядка следования членов функция (например: сумма, среднее, минимум и т.д), называемая функцией агрегации
* $\gamma$ и $\phi$ -- дифференцируемые функции, такие как перцептрон

Перенося на реалии ```GCNConv```:
* $\square$ есть $\sum_{j \in \mathcal{N}(i) \cup \{ i \}}$
* $\phi$ есть $\frac{1}{\sqrt{\hat d_i \hat d_j}}  \left( {\Theta} x_j \right)$
* $\gamma$ отсутствует

(<i>В самом же коде функции поменяют своё содержание ради простоты</i>)

Возвращаясь к классу ```MessagePassing```, грубо говоря, для каждой функции выше есть зарезервированные имена, которые нужно самому инициализировать при наследовании (как тот же ```forward()```). Методы класса ```MessagePassing```:
* ```MessagePassing.propagate(edge_index, size=None, **kwargs)``` -- метод вызывающий ```MessagePassing.message()``` и ```MessagePassing.update()```
* ```MessagePassing.message(...)``` -- то же что и $\phi$
* ```MessagePassing.update(aggr_out, ...)``` -- то же что и $\gamma$

для выбора функции агрегации ($\square$) на инициализации нужно будет указать параметр ```aggr=```

Для того чтобы создать ```GCNConv``` потребуется:
1. Добавить в матрицу смежности петель
2. Линейно преобразовать матрицу признаков
3. Посчитать коэфициенты нормализации -- $c(i,j)$
4. Применить коэфициенты нормализации
5. Суммировать значения соседних вершин

In [19]:
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

In [20]:
class GCN_udv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCN_udv, self).__init__(aggr='add')
        self.lin = torch.nn.Linear(in_channels, out_channels, bias=False)
        
    def forward(self, x, edge_index):        
        # Step 1
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        # Step 2
        x = self.lin(x)
        
        # Step 3
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        # Step 4-5
        return self.propagate(edge_index, x=x, norm=norm)
    
    def message(self, x_j, norm):
        
        # Step 4
        return norm.view(-1, 1) * x_j

Теперь попробуем вставить наш слой в самую первую модель:

In [21]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCN_udv(dataset.num_features, 16)
        self.conv2 = GCN_udv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [22]:
device='cuda'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)

best_val_acc = 0
for epoch in range(0, 150):
    
    model.train()
    optimizer.zero_grad()
    t = model(data)
    train_loss = F.nll_loss(t[data.train_mask], data.y[data.train_mask])
    train_loss.backward()
    optimizer.step()
    
    
    model.eval()
    accs = []
    for mask in [data.train_mask, data.val_mask]:
        pred = t[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    
    val_loss = F.nll_loss(t[data.val_mask], data.y[data.val_mask])
    train_acc, val_acc = accs
        
    if val_acc > best_val_acc:
        best_val_acc = val_acc
    
    if (epoch) % 10 == 0:
        log = 'Epoch: {:03d}, Train: {:.4f}, Train_loss: {:.4f} Val: {:.4f} Val_loss: {:.4f}'
        print(log.format(epoch, train_acc, train_loss, val_acc, val_loss))

Epoch: 000, Train: 0.1214, Train_loss: 1.9460 Val: 0.1540 Val_loss: 1.9450
Epoch: 010, Train: 0.9429, Train_loss: 0.9933 Val: 0.6640 Val_loss: 1.4260
Epoch: 020, Train: 0.9357, Train_loss: 0.3292 Val: 0.6800 Val_loss: 1.0647
Epoch: 030, Train: 0.9786, Train_loss: 0.1472 Val: 0.7200 Val_loss: 0.9840
Epoch: 040, Train: 1.0000, Train_loss: 0.0646 Val: 0.7320 Val_loss: 0.8916
Epoch: 050, Train: 0.9857, Train_loss: 0.0714 Val: 0.7380 Val_loss: 0.8796
Epoch: 060, Train: 0.9929, Train_loss: 0.0387 Val: 0.7300 Val_loss: 0.9120
Epoch: 070, Train: 1.0000, Train_loss: 0.0447 Val: 0.7220 Val_loss: 0.9112
Epoch: 080, Train: 1.0000, Train_loss: 0.0487 Val: 0.7460 Val_loss: 0.9167
Epoch: 090, Train: 1.0000, Train_loss: 0.0623 Val: 0.7240 Val_loss: 0.8935
Epoch: 100, Train: 1.0000, Train_loss: 0.0344 Val: 0.7340 Val_loss: 0.9100
Epoch: 110, Train: 1.0000, Train_loss: 0.0490 Val: 0.7340 Val_loss: 0.8824
Epoch: 120, Train: 1.0000, Train_loss: 0.0320 Val: 0.7400 Val_loss: 0.8830
Epoch: 130, Train: 0.9929

## Много математики и GNN 

### Чем примечательны графы? 

Начать нужно с особенностей данных — предполагается что графы не упорядочены — у них обычно нет начала и конца. Вершины же имеют свои признаки (features), поэтому когда мы пытаемя выразить матрицу в виде стандартного тензора (пока что будем считать что у описываемого графа нет ребер): 
$$ X = \begin{bmatrix}x_1 & x_2 & ... & x_n \end{bmatrix}^T  $$

Порядок уже появляется. Нам же важно, чтобы этот порядок не влиял на результат некоторой функции $f(X)$. Для того чтобы описать это математически введем матрицу перемешивания $P$. Как пример:
$$P_{(2,4,1,3)}X = \begin{bmatrix}0 & 1& 0 & 0 \\ 0 & 0& 0 & 1 \\ 1 & 0& 0 & 0\\ 0 & 0& 1 & 0\end{bmatrix} \begin{bmatrix}-x_1- \\ -x_2-\\ -x_3- \\ -x_4-\end{bmatrix} =\begin{bmatrix}-x_2- \\ -x_4-\\ -x_1- \\ -x_3-\end{bmatrix}$$

Количество таких матриц — факториал от количества вершин и для каждой должно выполняться условие:
$$ f(PX) = f(X) $$
Это есть <b>permutation invariance</b> (надеюсь появились флешбеки к прошлому разделу).

Но что если нам нам нужны предсказания на уровне каждой вершины? Результат функции должен быть также подвержен перестановке:
$$ f(PX) = Pf(X) $$
Это есть <b>permutation equivariance</b>


...это значит что мы можем рассматривать каждую строку отдельно как превращение признаков вершины в латентный вектор:
$$ h_i = \psi (x_i) \\ H = f(X) $$

Если же добавить инвариантность:

$$ f(X) = \phi (\bigoplus_{i \in V} \psi(x_i)) $$

$\bigoplus$ -- инвариантная функция агрегации

Добавим в граф смежную матрицу $A$, теперь функция принимает два параметра -- $f(X, A)$. Матрица $A$ симметрична, значит перемешивать нужно и колоны, и строки. Добиться этого можно с помощью $PAP^T$. В таком случае словия:
* Инвариантности: $f(PX, PAP^T) = f(X, A) $
* Эквариантности: $f(PX, PAP^T) = Pf(X,A) $

Добавление матрицы смежности значит добавление ценности связям вершин. Множество соседей описывается как:
$$\mathcal{N}_i = \{j : (i, j) \in E \vee (j, i) \in E\}$$
Тогда множество фич соседей:
$$X_{\mathcal{N}_i} = \{\{x_j : j \in \mathcal{N}_i\}\}$$
Определим локальную функцию $g(x_i, X_{\mathcal{N}_i})$ оперерирующую на этом множестве. В таком случае, как часть $f(X, A)$, она эквариантна:
$$f(X, A) = \begin{bmatrix}-g(x_0, X_{\mathcal{N}_0})- \\ -g(x_1, X_{\mathcal{N}_1})-\\ ... \\ -g(x_n, X_{\mathcal{N}_n})-\end{bmatrix}$$

<p>Теперь же попробуем определить $g$ и наконец таки выйдем на знакомую землю. $g$ (результат которой, я напомню, $h_i$) разделяется на несколько типов:</p>
<img src='./media/types_of_gnn.png'>

Разница между ними в том как считается коэфициент перед каждым соседом. Надеюсь вы узнали общую формулу ```MessagePassing```. В ```GCNConv```, раз это конволюционная сеть, коэфициент не является параметром для обучения, но в определенном смысле константа, хоть мы и выразили класс через ```MessagePassing```.

### Как использовать GNN? 

<p>По своей сути GNN преобразовывает признаки каждой вершины (беря в расчет признаки её соседа) в новые признаки и уже из них извлекает пользу, как показано ниже:</p>
<img src='./media/gnn_usage.png'>